In [2]:
pip install tabulate

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [45]:
import pandas as pd
import json
import sqlite3
import os
from tabulate import tabulate

# Function to load data based on file type (JSON or CSV)
def load_data(file_path, file_type):
    """
    Load data from a specified file.
    
    Parameters:
    - file_path: Path to the input file.
    - file_type: Type of the file ('csv' or 'json').
    
    Returns:
    - Loaded data as a pandas DataFrame (for CSV) or dictionary (for JSON).
    """
    try:
        if file_type == 'json':
            with open(file_path, 'r') as file:
                return json.load(file)
        elif file_type == 'csv':
            return pd.read_csv(file_path)
        else:
            raise ValueError("Unsupported file type")
    except Exception as e:
        print(f"Error loading data: {e}")
        return None

# Function to convert a JSON file into a CSV file
def json_to_csv(json_data, output_path):
    """
    Convert JSON data to CSV and save it to a file.
    
    Parameters:
    - json_data: JSON data to be converted.
    - output_path: Path to save the converted CSV file.
    """
    try:
        data = pd.json_normalize(json_data, 'battingSummary')
        data.to_csv(output_path, index=False)
        print(f"Converted JSON to CSV at {output_path}")
    except Exception as e:
        print(f"Error converting JSON to CSV: {e}")

# Function to convert a CSV file into a JSON file
def csv_to_json(csv_data, output_path):
    """
    Convert a CSV DataFrame to JSON and save it to a file.
    
    Parameters:
    - csv_data: Pandas DataFrame containing CSV data.
    - output_path: Path to save the converted JSON file.
    """
    try:
        csv_data.to_json(output_path, orient='records', indent=4)
        print(f"Converted CSV to JSON at {output_path}")
    except Exception as e:
        print(f"Error converting CSV to JSON: {e}")

# Function to save data to an SQLite database
def save_to_db(data, table_name, db_name='etl_data.db'):
    """
    Save pandas DataFrame to a SQLite database.
    
    Parameters:
    - data: Pandas DataFrame to be saved.
    - table_name: Name of the table in the SQLite database.
    - db_name: Name of the SQLite database file.
    """
    try:
        conn = sqlite3.connect(db_name)
        data.to_sql(table_name, conn, if_exists='replace', index=False)
        print(f"Saved data to table '{table_name}' in {db_name}")
        conn.close()
    except Exception as e:
        print(f"Error saving to database: {e}")

# Function to generate a summary of the data (records and columns)
def generate_summary(data):
    """
    Generate and print a summary of the DataFrame, including number of records and columns.
    
    Parameters:
    - data: Pandas DataFrame.
    
    Returns:
    - Tuple (number of records, number of columns).
    """
    if isinstance(data, pd.DataFrame):
        num_records = len(data)
        num_columns = len(data.columns)
        print(f"Summary:")
        print(f"Number of records: {num_records}")
        print(f"Number of columns: {num_columns}")
        print("Column names:")
        for col in data.columns:
            print(f"- {col}")
        return num_records, num_columns
    else:
        print("No summary available for this data format.")
        return 0, 0

# Function to display the first 5 rows of a SQLite table
def display_sqlite_table(db_name, table_name):
    """
    Display the first 5 rows of a specified table from a SQLite database.
    
    Parameters:
    - db_name: Name of the SQLite database.
    - table_name: Name of the table to display.
    """
    try:
        conn = sqlite3.connect(db_name)
        cursor = conn.cursor()
        cursor.execute(f"SELECT * FROM {table_name} LIMIT 5")
        rows = cursor.fetchall()
        headers = [description[0] for description in cursor.description]
        print(f"\nFirst 5 rows of table '{table_name}':")
        print(tabulate(rows, headers=headers, tablefmt="grid"))
        conn.close()
    except Exception as e:
        print(f"Error displaying SQLite table: {e}")

# Function to interactively add or delete columns from the dataset
def interactive_column_modification(data):
    """
    Interactive function to add or delete columns in a DataFrame. User can manually add specific data and delete columns until the 'done' option is inputted and the DataFrame is returned. 
    
    Parameters:
    - data: Pandas DataFrame to modify.
    
    Returns:
    - Modified DataFrame.
    """
    while True:
        action = input("Do you want to add or delete a column? (add/delete/done): ").lower()
        
        if action == 'add':
            col_name = input("Enter the name of the column to add: ")
            col_value = input("Enter the value for the new column: ")
            data[col_name] = col_value
            print(f"Added column '{col_name}' with value '{col_value}'")
        
        elif action == 'delete':
            col_name = input("Enter the name of the column to delete: ")
            if col_name in data.columns:
                data = data.drop(columns=[col_name])
                print(f"Deleted column '{col_name}'")
            else:
                print(f"Column '{col_name}' not found in the dataset")
        
        elif action == 'done':
            break
        
        else:
            print("Invalid action. Please enter 'add', 'delete', or 'done'.")
    
    return data

# Function to convert JSON to a pandas DataFrame
def json_to_dataframe(json_data):
    """
    Convert JSON data to a pandas DataFrame.
    
    Parameters:
    - json_data: JSON data to be converted.
    
    Returns:
    - Pandas DataFrame.
    """
    return pd.json_normalize(json_data, 'battingSummary')

# Function to convert a DataFrame back to JSON format
def dataframe_to_json(df):
    """
    Convert a pandas DataFrame to JSON.
    
    Parameters:
    - df: Pandas DataFrame to be converted.
    
    Returns:
    - JSON object.
    """
    return json.loads(df.to_json(orient='records'))

# Main ETL pipeline function to load, process, modify, and save data
def etl_pipeline(input_file, input_type, output_type, modify=True):
    """
    End-to-end ETL pipeline to load data, process, modify (if needed), and save to the desired output format.
    
    Parameters:
    - input_file: Input file path.
    - input_type: Type of the input file ('csv' or 'json').
    - output_type: Desired output format ('csv', 'json', or 'sql').
    - modify: Boolean indicating if interactive column modifications should be allowed.
    
    Returns:
    - Processed data in the desired format.
    """
    data = load_data(input_file, input_type)
    if data is None:
        return

    print("\n--- Data File Ingestion Summary ---")
    if input_type == 'json':
        data = json_to_dataframe(data)
    input_records, input_columns = generate_summary(data)

    if modify:
        print("\n--- Column Modification ---")
        data = interactive_column_modification(data)

    if output_type == 'csv':
        output_data = data
        output_data.to_csv('output.csv', index=False)
        print("Modified data saved to 'output.csv'")
    elif output_type == 'json':
        output_data = dataframe_to_json(data)
        with open('output.json', 'w') as f:
            json.dump(output_data, f, indent=4)
        print("Modified data saved to 'output.json'")
    elif output_type == 'sql':
        output_data = data
        table_name = os.path.splitext(os.path.basename(input_file))[0]
        save_to_db(output_data, table_name)
        display_sqlite_table('etl_data.db', table_name)

    print("\n--- Post Processing Summary ---")
    output_records, output_columns = generate_summary(data)

    print("\n--- Summary Comparison ---")
    print(f"Input:  {input_records} records, {input_columns} columns")
    print(f"Output: {output_records} records, {output_columns} columns")

    return output_data

# Main function to run the ETL pipeline
def main():
    """
    Main function to run the ETL pipeline. It prompts the user for input file type, output format, and runs the pipeline.
    """
    input_file = input("Enter the input file name: ")
    input_type = input("Enter the input file type (csv/json): ").lower()
    output_type = input("Enter the desired output type (csv/json/sql): ").lower()

    modified_data = etl_pipeline(input_file, input_type, output_type)

    if output_type == 'csv' and isinstance(modified_data, pd.DataFrame):
        modified_data.to_csv('output.csv', index=False)
        print("Modified data saved to 'output.csv'")
    elif output_type == 'json' and isinstance(modified_data, pd.DataFrame):
        modified_data.to_json('output.json', orient='records', indent=4)
        print("Modified data saved to 'output.json'")

# Execute the main function when the script is run
if __name__ == "__main__":
    main()



--- Data File Ingestion Summary ---
Summary:
Number of records: 699
Number of columns: 10
Column names:
- match
- teamInnings
- battingPos
- batsmanName
- dismissal
- runs
- balls
- 4s
- 6s
- SR

--- Column Modification ---
Added column 'wins' with value '10'
Modified data saved to 'output.csv'

--- Post Processing Summary ---
Summary:
Number of records: 699
Number of columns: 11
Column names:
- match
- teamInnings
- battingPos
- batsmanName
- dismissal
- runs
- balls
- 4s
- 6s
- SR
- wins

--- Summary Comparison ---
Input:  699 records, 10 columns
Output: 699 records, 11 columns
Modified data saved to 'output.csv'
